In [125]:
pip install konlpy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 19.4 MB 661 kB/s 
     |████████████████████████████████| 453 kB 60.9 MB/s 


In [5]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from konlpy.tag import Okt

## Task1

빈 칸을 채워주세요!

단계별 output이 github 파일에는 남아있으니 그 output과 동일한 형태인지 확인하면서 진행해주시면 됩니다~

In [56]:
#1. 생성할 문장 데이터

sentence = ("Brick walls are there for a reason and you must not think "
            "that the brick walls aren't there to keep us out, but rather "
            "in this way that the brick walls are there to show us how badly we want things.")

In [59]:
#2. 문자 집합 만들기
word_set = list(sorted(set(sentence)))

## 문제(1): 각 문자에 정수 인코딩 (공백도 하나의 원소로 포함)
vocab = {word:encode for encode, word in enumerate(word_set)}

In [60]:
print(vocab)

{' ': 0, "'": 1, ',': 2, '.': 3, 'B': 4, 'a': 5, 'b': 6, 'c': 7, 'd': 8, 'e': 9, 'f': 10, 'g': 11, 'h': 12, 'i': 13, 'k': 14, 'l': 15, 'm': 16, 'n': 17, 'o': 18, 'p': 19, 'r': 20, 's': 21, 't': 22, 'u': 23, 'w': 24, 'y': 25}


In [61]:
#3. 문자 집합 크기 확인

vocab_size = len(vocab)
print('문자 집합 크기 : {}'.format(vocab_size))

문자 집합 크기 : 26


In [62]:
#4. 하이퍼 파라미터 설정(자유롭게 수정해보세요!)
print(len(sentence))
hidden_size = vocab_size # 같아야 하는 것 확인!
sequence_length = 20  # 너무 길거나 너무 짧게 잡으면 안됩니다!
learning_rate = 0.01

198


In [63]:
#5. seqence 길이 단위 자르기

# 데이터 구성을 위한 리스트
x_data = []
y_data = []

## 문제(2): 반복문 내에서의 인덱싱을 사용하여 sequence_length 값 단위로 샘플을 잘라 데이터 만들기, y_str은 x_str은 한 칸씩 쉬프트된 sequnce

for i in range(0, len(sentence) - sequence_length):
  x_str = sentence[i:i+sequence_length]
  y_str = sentence[i+1:i+sequence_length+1]
  print(i, x_str, "->", y_str)

  # x_str과 y_str이 문자집합에 해당하는 인덱스를 각각 x_data, y_data에 append
  x_data.append([vocab[c] for c in x_str])
  y_data.append([vocab[d] for d in y_str])

0 Brick walls are ther -> rick walls are there
1 rick walls are there -> ick walls are there 
2 ick walls are there  -> ck walls are there f
3 ck walls are there f -> k walls are there fo
4 k walls are there fo ->  walls are there for
5  walls are there for -> walls are there for 
6 walls are there for  -> alls are there for a
7 alls are there for a -> lls are there for a 
8 lls are there for a  -> ls are there for a r
9 ls are there for a r -> s are there for a re
10 s are there for a re ->  are there for a rea
11  are there for a rea -> are there for a reas
12 are there for a reas -> re there for a reaso
13 re there for a reaso -> e there for a reason
14 e there for a reason ->  there for a reason 
15  there for a reason  -> there for a reason a
16 there for a reason a -> here for a reason an
17 here for a reason an -> ere for a reason and
18 ere for a reason and -> re for a reason and 
19 re for a reason and  -> e for a reason and y
20 e for a reason and y ->  for a reason and yo
21

In [64]:
# 출력해서 한 칸씩 쉬프트된 것 확인하기!

print(x_data[0])
print(y_data[0])

[4, 20, 13, 7, 14, 0, 24, 5, 15, 15, 21, 0, 5, 20, 9, 0, 22, 12, 9, 20]
[20, 13, 7, 14, 0, 24, 5, 15, 15, 21, 0, 5, 20, 9, 0, 22, 12, 9, 20, 9]


In [65]:
##6. 입력 시퀀스에 대해 원핫인코딩 수행

## 문제(4) : x_data를 원핫인코딩 > numpy의 eye를 쓸 수 있지 않을까?
x_one_hot = [np.eye(vocab_size,k=1,dtype=int)[i] for i in x_data]

##7. 입력 데이터, 레이블데이터 텐서로 변환

## 문제(5) : x_one_hot과 y_data 텐서로 변환 : 둘 다 같은 형식의 텐서로 변환하면 될까?? (FloatTensor, LongTesor 중 맞는 것은?)
X = torch.FloatTensor(x_one_hot)
Y = torch.LongTensor(y_data)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  ../torch/csrc/utils/tensor_new.cpp:201.)
  if __name__ == '__main__':


In [60]:
##8. 크기 확인
print('훈련 데이터의 크기 : {}'.format(X.shape))
print('레이블의 크기 : {}'.format(Y.shape))

훈련 데이터의 크기 : torch.Size([178, 20, 26])
레이블의 크기 : torch.Size([178, 20])


In [61]:
##9.원핫인코딩 결과 샘플 확인하기
print(X[0])

tensor([[0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 1., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0.,

In [62]:
##10. 레이블 데이터 샘플 확인하기
print(Y[0])

tensor([20, 13,  7, 14,  0, 24,  5, 15, 15, 21,  0,  5, 20,  9,  0, 22, 12,  9,
        20,  9])


In [87]:
##11. RNN 모델 구현

##문제(6) : 기본 pytorch 인자 넣기 연습 + forward 채우기
### 조건 : rnn layer 2개 쌓기 + 마지막은 fc layer
### batch_first 설정 필요할까? (유튜브 강의 참고)

class Net(torch.nn.Module):
  def __init__(self, input_dim, hidden_dim, layers):
    super(Net, self).__init__()
    self.rnn = torch.nn.RNN(input_dim, hidden_dim, num_layers=layers, batch_first=True)
    self.fc = torch.nn.Linear(hidden_dim, vocab_size, bias=True) # label개수 == vocab_size개수
  
  def forward(self, x):
    x,_ = self.rnn(x)
    x = self.fc(x)
    return x

In [95]:
net = Net(vocab_size, hidden_size, layers=2)

In [96]:
##12. loss function
criterion = torch.nn.CrossEntropyLoss()

##13. optimizer
optimizer = optim.Adam(net.parameters(), learning_rate)

In [97]:
##14. 출력 크기 점검
outputs = net(X)
print(outputs.shape)

torch.Size([178, 20, 26])


In [104]:
Y.view(-1).size()[0]

3560

In [106]:
##15. Training 시작

for i in range(100):
    optimizer.zero_grad()
    outputs = net(X)
    ##문제(7) : outputs, Y 형태 그대로 넣으면 안되죠. view 함수를 이용해 loss값을 계산해봅시다.
    loss = criterion(outputs.view(Y.view(-1).size()[0],-1), Y.view(-1))
    loss.backward()
    optimizer.step()

    #16. 예측결과 확인
    results = outputs.argmax(dim=2)
    predict_str = ""
    for j, result in enumerate(results):
        if j == 0: # 처음에는 예측 결과를 전부 가져오기
            predict_str += ''.join([world_set[t] for t in result])
        else: # 그 다음에는 마지막 글자만 반복 추가
            predict_str += world_set[result[-1]]

    print(predict_str)

bBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBB
bBBeefoBfeeeolffffnoefofoffhfofefofooffofffoBfofoofffefoffohfofoeffBBBeefoBfeeeolfoBfhfefoeffffefnfefffolfffBffooBffeeefofoffeeooffoffoffeffBBBeefoBfeeeolffffnoeffffefnohfBfohoofBffBfoofoBffffefoBe
toeeeeee eeeoe eeeeeeee eeeeeeeeeeeoeeeeh  eeeeeoheeeeeeeeeeeeeeeeeeeeeeeee eeeoe eheeeeeeeeeheeeeeeeeeo  eeeeeeeeeeeeeeeeeeeeeee eeeeeeeeeeeeeeeeee eeeoe eeeeeeeeheeeeoeeeeeh eeeeeeeeeeee eeeeeeee
te e ee   e     e  e      e e                       e     e    e   ee    e   e      e e  e  e               e   e    e e  e     e  e    e   ee    e   e     e  e  e  e       e   e             e    e
t                                                                                                                                                                                                    
t         

In [121]:
predict_str

"rick walls arenthere tor a reason and you must not think that the brick walls aren't there to keep us out, but rather in this way that the brick walls arenthere to show us how badly we want thinks,"

In [120]:
sentence

"Brick walls are there for a reason and you must not think that the brick walls aren't there to keep us out, but rather in this way that the brick walls are there to show us how badly we want things."

In [124]:
cor = 0
for i in range(len(predict_str)):
  if list(predict_str)[i] == list(sentence)[i+1]: # sentence의 경우 B가 없으므로
    cor +=1

print(f'정확도 {cor/len(predict_str)*100:.2f}%')

정확도 97.46%


결과가 어떤가요?? 마지막 에폭의 문장이 그럴싸한가요?

## Task2

위 sentence는 제가 임의로 생성한 문장들입니다.

마음에 드시는 문구 가져오셔서 문장이 어떻게 생성되는지 확인해보세요! 

영어가 아닌 한국어로 시도해보는 것도 좋겠죠? 

수정이 많이 필요(토큰화 등) 할 수 있으나 한번 시도해보시는 것 권장드립니다 :)

위 베이스라인은 어디든 수정하셔도 좋고 조금 더 자연스러운 문장이 나올 수 있게 다양한 시도를 해보세요!

조건 : 문장 3개 이상, 연결성이 있는 문장을 " " 으로 구분하여 ( )에 넣기

In [70]:
sentence = ('제게 대답하지 않으셔도 돼요 '
            '달래주지 않으셔도 '
            '다만 꺼지지 않는 작은 불빛이 '
            '여기 반짝 살아있어요 '
            '세상 모든 게 죽고 새로 태어나 '
            '다시 늙어갈 때에도 '
            '감히 이 마음만은 주름도 없이 '
            '여기 반짝 살아있어요 '
            '영영 살아있어요 '
            '영영 살아있어요 ')

In [71]:
okt = Okt()
word = okt.pos(sentence)
word

[('제', 'Noun'),
 ('게', 'Josa'),
 ('대답', 'Noun'),
 ('하지', 'Verb'),
 ('않으셔도', 'Verb'),
 ('돼요', 'Verb'),
 ('달래', 'Noun'),
 ('주지', 'Noun'),
 ('않으셔도', 'Verb'),
 ('다만', 'Noun'),
 ('꺼지지', 'Verb'),
 ('않는', 'Verb'),
 ('작은', 'Adjective'),
 ('불빛', 'Noun'),
 ('이', 'Josa'),
 ('여기', 'Noun'),
 ('반짝', 'Noun'),
 ('살아있어요', 'Verb'),
 ('세상', 'Noun'),
 ('모든', 'Noun'),
 ('게', 'Noun'),
 ('죽고', 'Verb'),
 ('새로', 'Adjective'),
 ('태어나', 'Verb'),
 ('다시', 'Noun'),
 ('늙어', 'Verb'),
 ('갈', 'Verb'),
 ('때', 'Noun'),
 ('에도', 'Josa'),
 ('감히', 'Noun'),
 ('이', 'Noun'),
 ('마음', 'Noun'),
 ('만은', 'Josa'),
 ('주름', 'Noun'),
 ('도', 'Josa'),
 ('없이', 'Adverb'),
 ('여기', 'Noun'),
 ('반짝', 'Noun'),
 ('살아있어요', 'Verb'),
 ('영영', 'Noun'),
 ('살아있어요', 'Verb'),
 ('영영', 'Noun'),
 ('살아있어요', 'Verb')]

In [72]:
word = [w[0] for w in word]
vocab = {word:encode for encode, word in enumerate(sorted(set(word)))}
vocab_size = len(vocab)
print(vocab_size)
vocab # 형태소 단위이기 때문에 뛰어쓰기 포함할 필요없음

34


{'갈': 0,
 '감히': 1,
 '게': 2,
 '꺼지지': 3,
 '늙어': 4,
 '다만': 5,
 '다시': 6,
 '달래': 7,
 '대답': 8,
 '도': 9,
 '돼요': 10,
 '때': 11,
 '마음': 12,
 '만은': 13,
 '모든': 14,
 '반짝': 15,
 '불빛': 16,
 '살아있어요': 17,
 '새로': 18,
 '세상': 19,
 '않는': 20,
 '않으셔도': 21,
 '없이': 22,
 '에도': 23,
 '여기': 24,
 '영영': 25,
 '이': 26,
 '작은': 27,
 '제': 28,
 '주름': 29,
 '주지': 30,
 '죽고': 31,
 '태어나': 32,
 '하지': 33}

In [73]:
hidden_size = vocab_size # 같아야 하는 것 확인!
sequence_length = 7 # 너무 길거나 너무 짧게 잡으면 안됩니다!
learning_rate = 0.01

#5. seqence 길이 단위 자르기

# 데이터 구성을 위한 리스트
x_data = []
y_data = []

## 문제(2): 반복문 내에서의 인덱싱을 사용하여 sequence_length 값 단위로 샘플을 잘라 데이터 만들기, y_str은 x_str은 한 칸씩 쉬프트된 sequnce

for i in range(0, len(word) - sequence_length):
  x_str = word[i:i+sequence_length]
  y_str = word[i+1:i+sequence_length+1]
  print(i, x_str, "->", y_str)

  # x_str과 y_str이 문자집합에 해당하는 인덱스를 각각 x_data, y_data에 append
  x_data.append([vocab[c] for c in x_str])
  y_data.append([vocab[d] for d in y_str])

0 ['제', '게', '대답', '하지', '않으셔도', '돼요', '달래'] -> ['게', '대답', '하지', '않으셔도', '돼요', '달래', '주지']
1 ['게', '대답', '하지', '않으셔도', '돼요', '달래', '주지'] -> ['대답', '하지', '않으셔도', '돼요', '달래', '주지', '않으셔도']
2 ['대답', '하지', '않으셔도', '돼요', '달래', '주지', '않으셔도'] -> ['하지', '않으셔도', '돼요', '달래', '주지', '않으셔도', '다만']
3 ['하지', '않으셔도', '돼요', '달래', '주지', '않으셔도', '다만'] -> ['않으셔도', '돼요', '달래', '주지', '않으셔도', '다만', '꺼지지']
4 ['않으셔도', '돼요', '달래', '주지', '않으셔도', '다만', '꺼지지'] -> ['돼요', '달래', '주지', '않으셔도', '다만', '꺼지지', '않는']
5 ['돼요', '달래', '주지', '않으셔도', '다만', '꺼지지', '않는'] -> ['달래', '주지', '않으셔도', '다만', '꺼지지', '않는', '작은']
6 ['달래', '주지', '않으셔도', '다만', '꺼지지', '않는', '작은'] -> ['주지', '않으셔도', '다만', '꺼지지', '않는', '작은', '불빛']
7 ['주지', '않으셔도', '다만', '꺼지지', '않는', '작은', '불빛'] -> ['않으셔도', '다만', '꺼지지', '않는', '작은', '불빛', '이']
8 ['않으셔도', '다만', '꺼지지', '않는', '작은', '불빛', '이'] -> ['다만', '꺼지지', '않는', '작은', '불빛', '이', '여기']
9 ['다만', '꺼지지', '않는', '작은', '불빛', '이', '여기'] -> ['꺼지지', '않는', '작은', '불빛', '이', '여기', '반짝']
10 ['꺼지지', '않는', '작은', '불빛', '이', '여기', 

In [74]:
# 출력해서 한 칸씩 쉬프트된 것 확인하기!

print(x_data[0])
print(y_data[0])

[28, 2, 8, 33, 21, 10, 7]
[2, 8, 33, 21, 10, 7, 30]


In [77]:
##6. 입력 시퀀스에 대해 원핫인코딩 수행

## 문제(4) : x_data를 원핫인코딩 > numpy의 eye를 쓸 수 있지 않을까?
x_one_hot = [np.eye(vocab_size,k=1,dtype=int)[i] for i in x_data]

##7. 입력 데이터, 레이블데이터 텐서로 변환

## 문제(5) : x_one_hot과 y_data 텐서로 변환 : 둘 다 같은 형식의 텐서로 변환하면 될까?? (FloatTensor, LongTesor 중 맞는 것은?)
X = torch.FloatTensor(x_one_hot)
Y = torch.LongTensor(y_data)

In [79]:
##8. 크기 확인
print('훈련 데이터의 크기 : {}'.format(X.shape))
print('레이블의 크기 : {}'.format(Y.shape))

훈련 데이터의 크기 : torch.Size([36, 7, 34])
레이블의 크기 : torch.Size([36, 7])


In [80]:
##9.원핫인코딩 결과 샘플 확인하기
print(X[0])

tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
        [0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0

In [81]:
##10. 레이블 데이터 샘플 확인하기
print(Y[0])

tensor([ 2,  8, 33, 21, 10,  7, 30])


In [82]:
##11. RNN 모델 구현

##문제(6) : 기본 pytorch 인자 넣기 연습 + forward 채우기
### 조건 : rnn layer 2개 쌓기 + 마지막은 fc layer
### batch_first 설정 필요할까? (유튜브 강의 참고)

class Net(torch.nn.Module):
  def __init__(self, input_dim, hidden_dim, layers):
    super(Net, self).__init__()
    self.rnn = torch.nn.RNN(input_dim, hidden_dim, num_layers=layers, batch_first=True)
    self.fc = torch.nn.Linear(hidden_dim, vocab_size, bias=True) # label개수 == vocab_size개수
  
  def forward(self, x):
    x,_ = self.rnn(x)
    x = self.fc(x)
    return x

In [83]:
net = Net(vocab_size, hidden_size, layers=2)

In [84]:
##12. loss function
criterion = torch.nn.CrossEntropyLoss()

##13. optimizer
optimizer = optim.Adam(net.parameters(), learning_rate)

In [85]:
##14. 출력 크기 점검
outputs = net(X)
print(outputs.shape)

torch.Size([36, 7, 34])


In [86]:
Y.view(-1).size()[0]

252

In [93]:
##15. Training 시작

for i in range(100):
    optimizer.zero_grad()
    outputs = net(X)
    ##문제(7) : outputs, Y 형태 그대로 넣으면 안되죠. view 함수를 이용해 loss값을 계산해봅시다.
    loss = criterion(outputs.view(Y.view(-1).size()[0],-1), Y.view(-1))
    loss.backward()
    optimizer.step()

    #16. 예측결과 확인
    results = outputs.argmax(dim=2)
    predict_str = "제게 "
    for j, result in enumerate(results):
        if j == 0: # 처음에는 예측 결과를 전부 가져오기
            predict_str += ''.join([word[t] for t in result])
        else: # 그 다음에는 마지막 글자만 반복 추가
            predict_str += ' '+word[result[-1]]

    print(predict_str)

제게 대답않으셔도주름죽고꺼지지주지이 죽고 돼요 하지 게 때 반짝 갈 다시 여기 살아있어요 모든 이 대답 마음 세상 만은 달래 않으셔도 제 않는 태어나 게 갈 작은 불빛 감히 다만 새로 다시 여기 살아있어요 늙어 살아있어요 늙어 살아있어요
제게 대답않으셔도주름죽고꺼지지주지이 죽고 돼요 하지 게 때 반짝 갈 다시 여기 살아있어요 모든 이 대답 마음 세상 만은 달래 않으셔도 제 않는 태어나 게 갈 작은 불빛 감히 다만 새로 다시 여기 살아있어요 늙어 살아있어요 늙어 살아있어요
제게 대답않으셔도주름죽고꺼지지주지이 죽고 돼요 하지 게 때 반짝 갈 다시 여기 살아있어요 모든 이 대답 마음 세상 만은 달래 않으셔도 제 않는 태어나 게 갈 작은 불빛 감히 다만 새로 다시 여기 살아있어요 늙어 살아있어요 늙어 살아있어요
제게 대답않으셔도주름죽고꺼지지주지이 죽고 돼요 하지 게 때 반짝 갈 다시 여기 살아있어요 모든 이 대답 마음 세상 만은 달래 않으셔도 제 않는 태어나 게 갈 작은 불빛 감히 다만 새로 다시 여기 살아있어요 늙어 살아있어요 늙어 살아있어요
제게 대답않으셔도주름죽고꺼지지주지이 죽고 돼요 하지 게 때 반짝 갈 다시 여기 살아있어요 모든 이 대답 마음 세상 만은 달래 않으셔도 제 않는 태어나 게 갈 작은 불빛 감히 다만 새로 다시 여기 살아있어요 늙어 살아있어요 늙어 살아있어요
제게 대답않으셔도주름죽고꺼지지주지이 죽고 돼요 하지 게 때 반짝 갈 다시 여기 살아있어요 모든 이 대답 마음 세상 만은 달래 않으셔도 제 않는 태어나 게 갈 작은 불빛 감히 다만 새로 다시 여기 살아있어요 늙어 살아있어요 늙어 살아있어요
제게 대답않으셔도주름죽고꺼지지주지이 죽고 돼요 하지 게 때 반짝 갈 다시 여기 살아있어요 모든 이 대답 마음 세상 만은 달래 않으셔도 제 않는 태어나 게 갈 작은 불빛 감히 다만 새로 다시 여기 살아있어요 늙어 살아있어요 늙어 살아있어요
제게 대답않으셔도주름죽고꺼지지주지이 죽고 돼요 하지 게 때 반짝 갈 다시 여기 살아있어요 모든 이 대답 마음 세상 만은 달래

In [94]:
predict_str

'제게 대답않으셔도주름죽고꺼지지주지이 죽고 돼요 하지 게 때 반짝 갈 다시 여기 살아있어요 모든 이 대답 마음 세상 만은 달래 않으셔도 제 않는 태어나 게 갈 작은 불빛 감히 다만 새로 다시 여기 살아있어요 늙어 살아있어요 늙어 살아있어요'

In [95]:
sentence

'제게 대답하지 않으셔도 돼요 달래주지 않으셔도 다만 꺼지지 않는 작은 불빛이 여기 반짝 살아있어요 세상 모든 게 죽고 새로 태어나 다시 늙어갈 때에도 감히 이 마음만은 주름도 없이 여기 반짝 살아있어요 영영 살아있어요 영영 살아있어요 '

In [102]:
cor = 0
for i in range(len(sentence)):
  if list(predict_str)[i] == list(sentence)[i]: # sentence의 경우 B가 없으므로
    cor +=1

print(f'정확도 {cor/len(predict_str)*100:.2f}%')
# 같은 로직을 거쳤지만 정확도가 많이 떨어진다.

정확도 11.36%
